https://boto3.amazonaws.com/v1/documentation/api/latest/guide/quickstart.html#configuration

https://docs.aws.amazon.com/code-samples/latest/catalog/code-catalog-python-example_code-textract.html

https://docs.aws.amazon.com/textract/latest/dg/what-is.html


In [42]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [45]:
%load_ext dotenv
%dotenv /home/jovyan/work/.env

In [11]:
# Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
# SPDX-License-Identifier: Apache-2.0

"""
Purpose

Shows how to use the AWS SDK for Python (Boto3) with Amazon Textract to
detect text, form, and table elements in document images.
"""

import json
import logging
import boto3
from botocore.exceptions import ClientError

logger = logging.getLogger(__name__)

# snippet-start:[python.example_code.textract.TextractWrapper]
class TextractWrapper:
    """Encapsulates Textract functions."""
    def __init__(self, textract_client, s3_resource, sqs_resource):
        """
        :param textract_client: A Boto3 Textract client.
        :param s3_resource: A Boto3 Amazon S3 resource.
        :param sqs_resource: A Boto3 Amazon SQS resource.
        """
        self.textract_client = textract_client
        self.s3_resource = s3_resource
        self.sqs_resource = sqs_resource
    # snippet-end:[python.example_code.textract.TextractWrapper]
    
    # snippet-start:[python.example_code.textract.StartDocumentAnalysis]
    def start_analysis_job(
            self, bucket_name, document_file_name, feature_types, sns_topic_arn,
            sns_role_arn):
        """
        Starts an asynchronous job to detect text and additional elements, such as
        forms or tables, in an image stored in an Amazon S3 bucket. Textract publishes
        a notification to the specified Amazon SNS topic when the job completes.
        The image must be in PNG, JPG, or PDF format.
        :param bucket_name: The name of the Amazon S3 bucket that contains the image.
        :param document_file_name: The name of the document image stored in Amazon S3.
        :param feature_types: The types of additional document features to detect.
        :param sns_topic_arn: The Amazon Resource Name (ARN) of an Amazon SNS topic
                              where job completion notification is published.
        :param sns_role_arn: The ARN of an AWS Identity and Access Management (IAM)
                             role that can be assumed by Textract and grants permission
                             to publish to the Amazon SNS topic.
        :return: The ID of the job.
        """
        try:
            response = self.textract_client.start_document_analysis(
                DocumentLocation={
                    'S3Object': {'Bucket': bucket_name, 'Name': document_file_name}},
                NotificationChannel={
                    'SNSTopicArn': sns_topic_arn, 'RoleArn': sns_role_arn},
                FeatureTypes=feature_types)
            job_id = response['JobId']
            logger.info(
                "Started text analysis job %s on %s.", job_id, document_file_name)
        except ClientError:
            logger.exception("Couldn't analyze text in %s.", document_file_name)
            raise
        else:
            return job_id
    # snippet-end:[python.example_code.textract.StartDocumentAnalysis]

In [62]:
if __name__ == "__main__":
    
    from dotenv import load_dotenv
    import os
    load_dotenv()

    bucket_name = 'textract-input-20221001'
    document_file_name=''
    ACCESS_KEY = os.getenv("ACCESS_KEY")
    SECRET_KEY = os.getenv("SECRET_KEY")

    textractClient = boto3.client('textract', 
                          region_name='us-east-1', 
                          aws_access_key_id=ACCESS_KEY,
                          aws_secret_access_key=SECRET_KEY)
    
    s3Resource = boto3.resource('s3',  
                      aws_access_key_id=ACCESS_KEY,
                      aws_secret_access_key=SECRET_KEY)
    
    # possible_feature_types = ['TABLES','FORMS','QUERIES']
    feature_types = ['TABLES']
    
    # tWrapper = TextractWrapper(textractClient,s3Resource)
    # tWrapper.start_analysis_job()

    your_bucket = s3.Bucket(bucket_name)

    extracted_data = []
    for s3_file in your_bucket.objects.all():
        print(s3_file)
        
        # use textract to process s3 file
        response = client.start_document_analysis(
                DocumentLocation={'S3Object': {'Bucket': bucket, 'Name': s3_file.key}},
                FeatureTypes=feature_types)

        job_id = response['JobId']
        logger.info("Started text analysis job %s on %s.", job_id, document_file_name)
        
        blocks=response['Blocks']

        for block in blocks:
                if block['BlockType'] != 'PAGE':
                    print('Detected: ' + block['Text'])
                    print('Confidence: ' + "{:.2f}".format(block['Confidence']) + "%")
                    
                    # Example case where you want to extract words with #
                    if("#" in block['Text']):
                        words = block['Text'].split()
                        for word in words:
                               if("#" in word):
                                    extracted_data.append({"word" : word, "file" : s3_file.key, "confidence": "{:.2f}".format(block['Confidence']) + "%"})
        
        # sleep 2 seconds to prevent ProvisionedThroughputExceededException
        sleep(2)

    df = pd.DataFrame(extracted_data)
    df = df.drop_duplicates()
    df.to_csv('output.csv')

s3.ObjectSummary(bucket_name='textract-input-20221001', key='example.pdf')


KeyError: 'Blocks'